In [ ]:
import torch
import time
from utils.utils import generate_mask, load_model, writeDACFile, sample_top_n
from dataloader.dataset import onehot, getNumClasses, class_name_to_int, int2classname
from utils.utils import interpolate_vectors

import os
import yaml

from DACTransformer.DACTransformer import TransformerDecoder
from DACTransformer.CondQueryTransformer import ClassConditionedTransformer
from DACTransformer.CondKeyTransformer import ClassConditionedKeyTransformer
from DACTransformer.PostNormCondDACTransformer import PostNormCondDACTransformerDecoder

import numpy as np
import matplotlib.pyplot as plt


In [ ]:
#For your reference:
classes=['pistons', 'wind', 'applause', 'bees']
print(f' ------- One hot vectors for classes ----------')
for i in range(4):
    print(f' {classes[i]} : \t{onehot(classes[i])}')

Morph over a vectors in vsequence lineary for (noramlized) time steps vtimes. Create your sequency:

In [ ]:
vsequence=[
    torch.tensor([1., 0., 0., 0., 0., 0., 0., 0.5]),
    torch.tensor([1., 0., 0., 0., 0., 0., 0., 0.5]),  
    torch.tensor([1., 0., 1., 0., 0., 0., 0., 0.5]),
    torch.tensor([1., 0., 1., 0., 0., 0., 0., 0.5]),
    torch.tensor([0., 0., 1., 0., 0., 0., 0., 0.5]),
    torch.tensor([0., 0., 1., 0., 0., 0., 0., 0.5])
]
vtimes=[0,.2,.4,.6,.8, 1] # must be the same length as the number of break points in vsequence
morphname='pistons.applause.doubling'

In [ ]:
### params
experiment_name = "01.20PostNormCond_2_" 
checkpoint_dir = 'runs' + '/' + experiment_name
cptnum =  600 #params['num_epochs'] # 300 #(must be in the checkpoint directory)

# Load YAML file
with open(checkpoint_dir + '/' + 'params.yaml', 'r') as file:
    params = yaml.safe_load(file)
    
TransformerClass =  globals().get(params['TransformerClass'])  
print(f"using TransformerClass = {params['TransformerClass']}") 
print(f' and TransformerClass is class object {TransformerClass}')

cond_size = 8 # num_classes + num params - not a FREE parameter!

embed_size = params['tblock_input_size'] -cond_size # 240 #32  # embed_size must be divisible by num_heads and by num tokens
print(f'embed_size is {embed_size}')


fnamebase='out' + '.e' + str(params['tblock_input_size']-cond_size) + '.l' + str(params['num_layers']) + '.h' + str(params['num_heads']) + '_chkpt_' + str(cptnum).zfill(4) 
checkpoint_path = checkpoint_dir + '/' +  fnamebase  + '.pth' 

# for saving sound 
outdir=checkpoint_dir
SAVEWAV=True

DEVICE='cpu' #####################################################''cuda'

inference_steps=86*20  #86 frames per second


# Values for interpolating the parameter value (start at minpval, up to maxpval, and then back)
minpval=0
maxpval=1
topn=1024 # sample from the top n logits


print(f'checkpoint_path = {checkpoint_path}, fnamebase = {fnamebase}' )

In [ ]:
if DEVICE == 'cuda' :
    torch.cuda.device_count()
    torch.cuda.get_device_properties(0).total_memory/1e9

    device = torch.device(DEVICE) # if the docker was started with --gpus all, then can choose here with cuda:0 (or cpu)
    torch.cuda.device_count()
    print(f'memeory on cuda 0 is  {torch.cuda.get_device_properties(0).total_memory/1e9}')
else :
    device=DEVICE
device

# Run Transformer to generate a DAC-coded audio file

In [ ]:
 
def inference(model, inference_cond, Ti_context_length, vocab_size, num_tokens, inference_steps, fname) :
    model.eval()
    mask = generate_mask(Ti_context_length, Ti_context_length).to(device)

    # The "input data" is random with a sequence length equal to the context length (and the mask) which is used 
    # to generate the first step of the output.It is not included in the output.
    input_data = torch.randint(0, vocab_size, (1, Ti_context_length, num_tokens)).to(device)  # Smaller context window for inference
    #Extend the first conditional vector to cover the "input" which is of length Ti_context_length
    inference_cond = torch.cat([inference_cond[:, :1, :].repeat(1, Ti_context_length, 1), inference_cond], dim=1)
    predictions = []

    print(f' in the inference function, the shape of input_data is {input_data.shape} and the shape of the inference_cond is {inference_cond.shape}')

    t0 = time.time()
    for i in range(inference_steps):  # 
        if cond_size == 0:
            output = model(input_data, None, mask) # step through 
        else : 
            output = model(input_data, inference_cond[:, i:Ti_context_length+i, :], mask) # step through

        # This takes the last vector of the sequence (the new predicted token stack) so has size(b,steps,4,1024)
        # This it takes the max across the last dimension (scores for each element of the vocabulary (for each of the 4 tokens))
        # .max returns a duple of tensors, the first are the max vals (one for each token) and the second are the
        #        indices in the range of the vocabulary size. 
        # THAT IS, the selected "best" tokens (one for each codebook) are taken independently
        ########################### next_token = output[:, -1, :, :].max(-1)[1]  # Greedy decoding for simplicity
        next_token = sample_top_n(output[:, -1, :, :],2) # top 1 would be the same as max in the comment line above
            
        #print(f'next_token: {next_token} which had a top 1 logit value of {sample_top_n(output[:, -1, :, :],1)}')
        #print(f'                                       and a had a top 2 logit value of {sample_top_n(output[:, -1, :, :],2)}')
                                                                           
                                                                           
        
        predictions.append(next_token)
        input_data = torch.cat([input_data, next_token.unsqueeze(1)], dim=1)[:, 1:]  # Slide window

    t1 = time.time()
    inf_time = t1-t0
    print(f'inference time for {inference_steps} steps, or {inference_steps/86} seconds of sound is {inf_time}' )

    dacseq = torch.cat(predictions, dim=0).unsqueeze(0).transpose(1, 2)
    if mask == None:
        writeDACFile(fname + '_unmasked', dacseq)
    else :
        writeDACFile(fname, dacseq)   

    print(f'dacseq shape written to file is of shape {dacseq.shape}')

In [ ]:
print(f' About to call load_model with TransformerClass = {TransformerClass}')
model, Ti_context_length, vocab_size, num_codebooks, cond_size = load_model(checkpoint_path,  TransformerClass, DEVICE)
print(f'Mode loaded, context_length (Ti_context_length) = {Ti_context_length}')
# Count the number of parameters
num_params = sum(p.numel() for p in model.parameters())
print(f'Total number of parameters: {num_params}')


model.to(device);


In [ ]:

for snum in range(1) : 
    if cond_size == 0 :
        inference_cond = None
    else : 

        inference_cond=interpolate_vectors(vsequence, [round(x * inference_steps) for x in vtimes]) #length must cover staring context window+inf steps

        
        # Make a picture --------------------------------------------------------------
        # Extract the 2D array of shape [n, m]
        data = inference_cond[0]
        # Find components that change over time
        changing_indices = [i for i in range(cond_size) if not torch.all(data[:, i] == data[0, i])]

        # Plot the changing components
        plt.figure(figsize=(10, 3))

        for i in changing_indices:
            if i != 7 :
                plt.plot(data[:, i], label=f'{int2classname[i]} ({i})')
            else : 
                plt.plot(data[:, i], label=f'Parameter ({i})', linestyle='--')

        plt.xlabel('Time Steps')
        plt.ylabel('Component Values')
        plt.title(f' {morphname}')
        plt.legend()
        plt.grid(True)
        plt.show()

        inference_cond=inference_cond.to(device)
        print(f'shape of inf_cond is  = {inference_cond.shape}') 

        
        outfname=outdir+"/"+ "dacs" + "/" +  morphname + '_chkpt_' + str(cptnum).zfill(4) +  "_steps_"+str(inference_steps).zfill(4) +'.topn_'+ f"{topn:04d}"
        print(f'outfname is {outfname}')
        inference(model, inference_cond, Ti_context_length, vocab_size, num_codebooks, inference_steps, outfname ) 

print('Done')

<div style="width: 100%; height: 20px; background-color: black;"></div>

# Read the dacfile, decode it to audio, and display

In [ ]:
import dac
import soundfile as sf
import IPython.display as ipd

In [ ]:
# The first time you do this, it can take a while. Go get coffee. After that, it uses a cached version
dacmodel_path = dac.utils.download(model_type="44khz") 
print(f'The DAC decoder is in {dacmodel_path}')
with torch.no_grad():
    dacmodel = dac.DAC.load(dacmodel_path)

    dacmodel.to(device); #wanna see the model? remove the semicolon
    dacmodel.eval();  # need to be "in eval mode" in order to set the number of quantizers

In [ ]:
# --------  derived ------ don't change these 
selected_file=outfname + ".dac"
print(f' selected_file is {selected_file}')

with torch.no_grad():
    dacfile = dac.DACFile.load(selected_file)
    # FIRST - Decompress it back to an AudioSignal\ from codes to z (1024) to signal   
    print(f'dacfile.codes shape is: {dacfile.codes.shape}')
    t0=time.time()
    asig=dacmodel.decompress(dacfile)
    t1=time.time()
    
    inf_time = t1-t0
    print(f'decompress time for {asig.audio_data.shape[2]/44100} seconds of sound is {inf_time}' )
    print(f'asig.audio_data.shape[2] is {asig.audio_data.shape[2]}')
    
    asig.cpu().widget()
    asig.save_image(outfname + ".jpg")
    asig.audio_data.shape

In [ ]:
adata = asig.samples.view(-1).numpy()
if SAVEWAV :  
    sf.write(outfname + ".wav", adata, 44100)

In [ ]:


# Audio display
plt.plot(adata)
# Audio player
ipd.Audio(adata, rate=44100) 